In [23]:
import pandas as pd
import numpy as np
import sklearn.metrics.pairwise as pairwise

movies_metadata = pd.read_csv('./movies_metadata.csv')
ratings_small = pd.read_csv('./ratings_small.csv')
keywords = pd.read_csv('./keywords.csv')


/Users/williamcxd7/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [25]:
ratings_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [26]:
ratings_small.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [27]:
print(ratings_small.userId.nunique())
print(ratings_small.movieId.nunique()) 

671
9066


In [28]:
ratings_small.duplicated(subset=['userId','movieId']).sum() 
# Thus There is no need for the data cleaning for ratings_small
# We could directly use the data of ratings_small

0

In [51]:
from sklearn.model_selection import train_test_split
train_data,test_data =train_test_split(ratings_small,test_size=0.3) 

In [52]:
m_users = ratings_small.userId.nunique()
n_items = ratings_small.movieId.nunique()

In [53]:
train_data

,userId,movieId,rating,timestamp
76081,529,994,5.0,960087034
42742,306,1682,4.0,939722727
96623,647,541,5.0,947292496
32095,232,2393,4.0,955087343
81830,558,3869,3.0,967385125
...,...,...,...,...
35979,261,7143,4.0,1101665980
49003,358,1353,1.0,957479762
40186,294,235,3.0,1082753766
24839,178,44195,4.0,1437424646


In [54]:
test_data

,userId,movieId,rating,timestamp
37314,268,3256,3.0,1314888511
23496,165,4734,2.5,1111482568
69243,480,3948,5.0,1272668838
61372,450,112552,4.0,1449349696
58330,423,38061,3.5,1353703019
...,...,...,...,...
5531,30,2474,5.0,945294381
30161,213,111659,3.5,1462638170
15353,102,14,4.0,957895013
67472,472,1250,5.0,939003828


In [55]:
ratings_small_sub = ratings_small.iloc[:,[0,1,2]]

In [56]:
ratings_small_sub

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [57]:
# Get ratings matrix in the form of pivot dataframe from the above dataframe
new_ratings_pivot = ratings_small_sub.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [58]:
# Fill the test data positions with zeros so that the ratings 
# matrix of pivot dataframe contain only train data
new_ratings_pivot_train = new_ratings_pivot.copy()
for userId, movieId,_,_ in test_data.values:
    new_ratings_pivot_train.loc[int(userId),int(movieId)] = 0

In [59]:
# Making it to the matrix form
ratings_small_matrix = np.array(new_ratings_pivot_train)

In [60]:
ratings_small_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [61]:
# obtain the cosine similarity matrix of each movies
from sklearn.metrics.pairwise import pairwise_distances
item_similarity_m = pairwise_distances(ratings_small_matrix.T,metric='cosine')

In [62]:
item_similarity_m

array([[0.        , 0.75882582, 0.81450611, ..., 1.        , 1.        ,
        0.9317    ],
       [0.75882582, 0.        , 0.80699999, ..., 1.        , 1.        ,
        1.        ],
       [0.81450611, 0.80699999, 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.9317    , 1.        , 1.        , ..., 1.        , 1.        ,
        0.        ]])

In [63]:
# similarly, get the test data matrix by erasing the train data
# in the original data matrix
new_ratings_pivot_test = new_ratings_pivot.copy()
for userId, movieId,_,_ in train_data.values:
    new_ratings_pivot_test.loc[int(userId),int(movieId)] = 0
ratings_small_matrix_test = np.array(new_ratings_pivot_test)

In [64]:
# Get the RMSE of Test Data Set
item_prediction = ratings_small_matrix_test.dot(item_similarity_m) / np.array(np.abs(item_similarity_m).sum(axis=1))

from sklearn.metrics import mean_squared_error
from math import sqrt
prediction_flatten = user_item_precdiction[ratings_small_matrix_test.nonzero()]
test_data_matrix_flatten = ratings_small_matrix_test[ratings_small_matrix_test.nonzero()]
error_test = sqrt(mean_squared_error(prediction_flatten,test_data_matrix_flatten))
print('RMSE of Test Data Set',error_test)

RMSE of Test Data Set 3.5779498728085
